In [91]:
# Imports
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn import ensemble
from sklearn import svm
import time
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import math
import matplotlib.ticker as plticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics 
from sklearn.ensemble import AdaBoostClassifier
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.random_projection import GaussianRandomProjection
from sklearn.decomposition import FactorAnalysis

In [92]:
class Game:
    winner = ""
    loser = ""
    date = ""
    winner_rank = 0
    loser_rank = 0
    def __init__(self, winner, loser, date, winner_rank, loser_rank):
        self.winner = winner
        self.loser = loser
        self.date = date
        self.winner_rank = winner_rank
        self.loser_rank = loser_rank
    def if_winner(self, player):
        return self.winner == player
    def if_loser(self, player):
        return self.loser == player
    def if_player(self, player):
        return self.if_winner(player) or self.if_loser(player)
    def get_opponent(self, player):
        return self.loser if self.winner == player else self.winner
    def __str__(self):
        return f"{self.winner} beat {self.loser} on {self.date}"
    
    # a function to sort
    def __cmp__(self, other):
        return self.date < other.date
    def __lt__(self, other):
        return self.date < other.date

class Player:
    def __init__(self, name):
        self.name = name
        self.games = []
        self.rankings = []
    def ranking(self):
        return np.mean(self.rankings)
    def add_ranking(self, ranking):
        # if nan ignore
        if math.isnan(ranking):
            return
        self.rankings.append(ranking)
    def add_game(self, game):
        self.games.append(game)
        if game.if_winner(self.name):
            self.add_ranking(game.winner_rank)
        elif game.if_loser(self.name):
            self.add_ranking(game.loser_rank)
    
    def remove_game(self, game):
        self.games.remove(game)

    def __cmp__(self, other):
        return len(self.games) > len(other.games)
    def __lt__(self, other):
        return len(self.games) > len(other.games)
    def extract_games(self, player):
        return [game for game in self.games if game.if_player(player)]
    def isBetterThan(self, player):
        count = 0
        for game in self.games:
            if game.if_winner(self.name) and game.if_loser(player.name):
                count += 1
            if game.if_loser(self.name) and game.if_winner(player.name):
                count -= 1
        return count
    def __str__(self):
        return f"{self.name}"


In [93]:
games = []
players = {}
# all files in ./data
for file in os.listdir("./data"):
    #open file
    with open("./data/" + file) as f:
        # read csv
        df = pd.read_csv(f)
        # iterate through rows
        for index, row in df.iterrows():
            games.append(Game(row["winner_name"], row["loser_name"], row["tourney_date"], row["winner_rank"], row["loser_rank"]))
            if row["winner_name"] not in players:
                players[row["winner_name"]] = Player(row["winner_name"])
            if row["loser_name"] not in players:
                players[row["loser_name"]] = Player(row["loser_name"])
            players[row["winner_name"]].add_game(games[-1])
            players[row["loser_name"]].add_game(games[-1])
# sort games by date
games.sort()
# sort players by number of games
players = sorted(players.values())

In [94]:
print(players)
for player in players[:30]:
    print(player.name, len(player.games), player.ranking())

[<__main__.Player object at 0x7ff6b945a7f0>, <__main__.Player object at 0x7ff6b92eda00>, <__main__.Player object at 0x7ff6e111bbe0>, <__main__.Player object at 0x7ff6b9365220>, <__main__.Player object at 0x7ff6b2403460>, <__main__.Player object at 0x7ff6b93afac0>, <__main__.Player object at 0x7ff6c12ded30>, <__main__.Player object at 0x7ff6e081cb20>, <__main__.Player object at 0x7ff6b24037c0>, <__main__.Player object at 0x7ff6a0b40100>, <__main__.Player object at 0x7ff6b9167a60>, <__main__.Player object at 0x7ff6b98bbbb0>, <__main__.Player object at 0x7ff6e11a5100>, <__main__.Player object at 0x7ff6b2403eb0>, <__main__.Player object at 0x7ff6b9275910>, <__main__.Player object at 0x7ff6b2277eb0>, <__main__.Player object at 0x7ff6b93ceb80>, <__main__.Player object at 0x7ff6b22dcd60>, <__main__.Player object at 0x7ff6b9445100>, <__main__.Player object at 0x7ff6b276d070>, <__main__.Player object at 0x7ff710931580>, <__main__.Player object at 0x7ff6e0c4c790>, <__main__.Player object at 0x7f

In [95]:
def calc_top_players(top):
    most_played_players = players[:top]
    # for player in most_played_players:
    #     print(player.name, len(player.games))
    count = 0
    for i in range(0, top):
        for j in range (i + 1, top):
            if len(players[i].extract_games(players[j].name)) > 0:
                count += 1
    print(count)
    print(f"total: ", count / len(most_played_players) / (len(most_played_players) - 1) * 2)


In [96]:
def generate_relation(top):
    # save in a file 
    n = len(players)
    print(f"original n: {n}")
    selected = players.copy()
    while (n > top):
        n = max (n // 2, top)
        print(f"n: {n}")
        selected = selected[:n]
        for player in selected:
            for game in player.games:
                if any([game.if_player(p.name) for p in selected]):
                    continue
                else:
                    player.remove_game(game)
        selected = sorted(selected)
    
    for player in selected:
        print(player.name, len(player.games), player.ranking())
    
    count = 0
    for i in range(0, top):
        for j in range (i + 1, top):
            if len(selected[i].extract_games(selected[j].name)) > 0:
                count += 1
    print(f"ratio: ", count / top / (top - 1) * 2)


    relation = np.zeros((top, top))


    for i in range(0, top):
        relation[i][i] = 1
        for j in range (i + 1, top):
            if len(players[i].extract_games(players[j].name)) > 0:
                isBetter = players[i].isBetterThan(players[j])
                # print(f"{players[i].name} is better than {players[j].name}: {isBetter}, rankings {players[i].ranking()} {players[j].ranking()}")
                relation[i][j] = isBetter > 0
                relation[j][i] = not relation[i][j]
            else:
                # randomly choose 0 or 1
                # relation[i][j] = random.randint(0, 1)
                # relation[j][i] = not relation[i][j]
                relation[i][j] = (players[i].ranking() < players[j].ranking())
                relation[j][i] = not relation[i][j]

    wrong_edges = 0
    for i in range(0, top):
        for j in range (i + 1, top):
            if relation[i][j] != (players[i].ranking() < players[j].ranking()):
                wrong_edges += 1
    print(f"{wrong_edges} out of {top * (top - 1) / 2} edges are wrong")
    
    return relation
                


In [98]:
def to_file (relation, file_name):
    with open(file_name, "w") as f:
        f.write(str(len(relation)) + "\n")
        for i in range(0, len(relation)):
            for j in range(0, len(relation)):
                f.write(str(int(relation[i][j])))
                if j != len(relation) - 1:
                    f.write(" ")
            f.write("\n")

In [97]:
a = generate_relation(30)

original n: 2462
n: 1231
n: 615
n: 307
n: 153
n: 76
n: 38
n: 30
Roger Federer 1275 5.921739130434783
David Ferrer 973 17.144628099173552
Rafael Nadal 936 9.487124463519313
Tommy Robredo 862 26.756440281030446
Novak Djokovic 842 10.69833729216152
Mikhail Youzhny 829 36.909535452322736
Andy Roddick 828 17.349090909090908
Tomas Berdych 827 16.185230024213077
Nikolay Davydenko 822 27.82076637824475
Lleyton Hewitt 794 29.748403575989784
Feliciano Lopez 759 40.39337748344371
Jarkko Nieminen 757 49.8814913448735
Fernando Verdasco 739 32.50407608695652
Andy Murray 727 14.741403026134801
Juan Carlos Ferrero 717 24.245467224546722
Tommy Haas 699 57.97826086956522
Ivan Ljubicic 690 32.69001490312966
Jurgen Melzer 675 51.5089552238806
Richard Gasquet 673 30.716840536512667
Radek Stepanek 658 47.1
Philipp Kohlschreiber 653 43.52760736196319
Stanislas Wawrinka 631 28.100158982511925
Gilles Simon 629 27.941176470588236
James Blake 622 45.61414790996785
Nicolas Almagro 611 35.55482815057283
Andreas Se

In [101]:
to_file(a, "../data/tennis_relation.txt")

In [30]:
generate_relation(100)

2013 out of 4950.0 edges are wrong


array([[1., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 1., 0., 1.],
       [1., 1., 1., ..., 0., 1., 1.],
       ...,
       [1., 0., 1., ..., 1., 0., 1.],
       [1., 1., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [129]:
calc_top_players(100)

4062
total:  0.8206060606060606


In [130]:
calc_top_players(200)

11028
total:  0.5541708542713568
